# Chapter 6

### 6.1 Reading and Writing Data in Text Format



In [1]:
import pandas as pd
import sys
import numpy as np
import csv
import json
from lxml import objectify

In [2]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
names = ['a', 'b', 'c', 'd', 'message']

In [9]:
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [11]:
parsed = pd.read_csv('examples/csv_mindex.csv',index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [14]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [15]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [17]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [19]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


### Different NA sentinels can be specified for each column in a dict:


In [20]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}


In [21]:
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

In [22]:
pd.options.display.max_rows = 10

In [23]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [24]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [25]:
### To read a file in pieces, specify a chunksize as a number of rows:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

In [26]:
### The TextParser object returned by read_csv allows you to iterate over the parts of the file according to the chunksize. For example, we can iterate over ex6.csv, aggre‐ gating the value counts in the 'key' column like so:

In [27]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [28]:
tot = pd.Series([]) 


/var/folders/26/nhlrw59x2f1d6s6ljr___mm40000gp/T/ipykernel_55461/3246538380.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


In [29]:
for piece in chunker:
        tot = tot.add(piece['key'].value_counts(), fill_value=0)

In [30]:
tot = tot.sort_values(ascending=False)

In [31]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format

In [32]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [33]:
### Using DataFrame’s to_csv method, we can write the data out to a comma-separated file:

In [34]:
data.to_csv('examples/out.csv')
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [35]:
import sys


In [36]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [37]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [38]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [39]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [40]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv')
!cat examples/tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### Working with Delimited Formats

In [41]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [42]:
import csv

In [43]:
f = open('examples/ex7.csv')
reader = csv.reader(f)

In [44]:
for line in reader:print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [45]:
with open('examples/ex7.csv') as f:lines = list(csv.reader(f))

In [46]:
header, values = lines[0], lines[1:]

In [47]:
data_dict = {h: v for h, v in zip(header, zip(*values))}


In [48]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [49]:
class my_dialect(csv.Dialect): 
    lineterminator = '\n' 
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL


In [50]:
reader = csv.reader(f, dialect=my_dialect)

ValueError: I/O operation on closed file.

In [ ]:
reader = csv.reader(f, delimiter='|')

In [ ]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect) 
    writer.writerow(('one', 'two', 'three')) 
    writer.writerow(('1', '2', '3')) 
    writer.writerow(('4', '5', '6')) 
    writer.writerow(('7', '8', '9'))

### JSON (Java Script Object Notation) Data

In [ ]:
obj = """
    {"name": "Wes",
     "places_lived": ["United States", "Spain", "Germany"],
     "pet": null,
     "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
                  {"name": "Katie", "age": 38,
                   "pets": ["Sixes", "Stache", "Cisco"]}]
} """

In [ ]:
import json

In [ ]:
result = json.loads(obj)
result

In [ ]:
asjson = json.dumps(result)

In [ ]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

In [ ]:
!cat examples/example.json

In [ ]:
data = pd.read_json('examples/example.json')
data

In [ ]:
print(data.to_json())

In [ ]:
print(data.to_json(orient='records'))

### XML and HTML: Web Scraping

In [51]:
conda install lxml


NotWritableError: The current user does not have write permissions to a required path.
  path: /Applications/JupyterLab.app/Contents/Resources/jlab_server/pkgs/cache/b89cf7bf.json
  uid: 502
  gid: 20

If you feel that permissions on this path are set incorrectly, you can manually
change them by executing

  $ sudo chown 502:20 /Applications/JupyterLab.app/Contents/Resources/jlab_server/pkgs/cache/b89cf7bf.json

In general, it's not advisable to use 'sudo conda'.



Note: you may need to restart the kernel to use updated packages.


In [52]:
!pip install beautifulsoup4 html5lib

Defaulting to user installation because normal site-packages is not writeable


In [53]:
!pip install lxml

Defaulting to user installation because normal site-packages is not writeable


In [54]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [55]:
len(tables)

1

In [56]:
failures = tables[0]

In [57]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [58]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
    

In [59]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

### Parsing XML with lxml.objectify

In [60]:
!pip3 install lxml  

Defaulting to user installation because normal site-packages is not writeable


In [61]:
from lxml import objectify

In [62]:
path = 'examples/mta_perf/Performance_MNR.xml'


In [63]:
parsed = objectify.parse(open(path))
   

FileNotFoundError: [Errno 2] No such file or directory: 'examples/mta_perf/Performance_MNR.xml'

In [ ]:
root = parsed.getroot()

In [ ]:
data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ','DESIRED_CHANGE', 'DECIMAL_PLACES']


In [ ]:
for elt in root.INDICATOR: 
    el_data = {}
    for child in elt.getchildren(): 
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = pd.DataFrame(data)


In [ ]:
perf.head()

In [ ]:
from io import StringIO

In [ ]:
tag = '<a href="http://www.google.com">Google</a>'

In [ ]:
root = objectify.parse(StringIO(tag)).getroot()

In [ ]:
root

In [ ]:
root.get('href')

In [ ]:
 root.text

### 6.2 Binary Data Formats

In [ ]:
import pandas as pd

In [ ]:
!pip3 install tables


In [ ]:
!pip3 install openpyxl

In [64]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [65]:
frame.to_pickle('examples/frame_pickle')

In [66]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [67]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [68]:
store = pd.HDFStore('mydata.h5')

In [69]:
store['obj1'] = frame

In [70]:
store['obj1_col'] = frame['a']


In [71]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [72]:
store['obj1']

,a
0,0.063821
1,1.171857
2,1.075569
3,-0.213914
4,-2.474230
...,...
95,-1.225285
96,-1.279350
97,1.009437
98,0.729932


In [73]:
store.put('obj2', frame, format='table')


In [74]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-0.231545
11,-0.099746
12,-1.090730
13,-1.399326
14,-0.166886
15,1.722184


In [75]:
store.close()

In [76]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [77]:
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])
    

,a
0,0.063821
1,1.171857
2,1.075569
3,-0.213914
4,-2.474230


### Reading Microsoft Excel Files

In [78]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [79]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [80]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')

In [81]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [82]:
writer = pd.ExcelWriter('examples/ex2.xlsx')


In [83]:
frame.to_excel(writer, 'Sheet1')

In [84]:
writer.save()

In [85]:
frame.to_excel('examples/ex2.xlsx')

### 6.3 Interacting with Web APIs

In [86]:
import requests

In [87]:
 url = 'https://api.github.com/repos/pandas-dev/pandas/issues'


In [88]:
resp = requests.get(url)


In [89]:
resp

<Response [200]>

In [91]:
data = resp.json()


In [92]:
data[0]['title']

'CI: Simplify call to asv'

In [93]:
issues = pd.DataFrame(data, columns=['number', 'title','labels', 'state'])

In [94]:
issues

,number,title,labels,state
0,46599,CI: Simplify call to asv,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
1,46598,CI: Add durations to benchmarks build,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
2,46597,Make test_localization tests execute on window...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,46596,BUG: custom callable in to_sql stopped working...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,46595,BUG: `tm.set_locale` does not correctly set ba...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
...,...,...,...,...
25,46569,BUG: ENH: Allow callable for on_bad_lines in ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,46568,CLN/TYP: assorted,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
27,46565,TYP: setter for index/columns property-like (A...,"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open
28,46564,Don't run GHA CI for Azure changes and vice-versa,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open


### 6.4 Interacting with Databases

In [95]:
import sqlite3

In [96]:
query = """ CREATE TABLE test(a VARCHAR(20), b VARCHAR(20),c REAL,d INTEGER);"""

In [97]:
con = sqlite3.connect('mydata.sqlite')
con.execute(query)

In [98]:
con.commit()

In [99]:
### Then, insert a few rows of data:
data = [('Atlanta', 'Georgia', 1.25, 6),('Tallahassee', 'Florida', 2.6, 3),('Sacramento', 'California', 1.7, 5)]

In [100]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [101]:
con.executemany(stmt, data)

In [102]:
con.commit()

In [ ]:
### Most Python SQL drivers (PyODBC, psycopg2, MySQLdb, pymssql, etc.) return a list of tuples when selecting data from a table:

In [103]:
cursor = con.execute('select * from test')

In [104]:
rows = cursor.fetchall()

In [106]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [107]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [108]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [113]:
!pip3 install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 KB 4.9 MB/s eta 0:00:00


In [114]:
import sqlalchemy as sqla

In [115]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [117]:
pd.read_sql('select * from test', db)


,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [ ]:
###### completed Chapter 6.  #######